## Problem 1: Modified Gram-Schmidt

In [59]:
#Import libraries and initialize a random matrix A

import numpy as np
from scipy import linalg as la

A = np.random.random((4,2))
print(A)

[[ 0.70192829  0.12393982]
 [ 0.56785236  0.95715545]
 [ 0.09677499  0.29348275]
 [ 0.18955045  0.04483686]]


In [60]:
def gram(A):
    """Modified Gram-Schmidt algorithm; takes in matrix A and gives QR decomposition"""
    m = np.shape(A)[0]
    n = np.shape(A)[1]
    Q = np.copy(A)
    R = np.zeros((n, n))
    i = 0
    while i <= n-1:
        a = Q[:,i]
        R[i][i] = np.linalg.norm(a)
        Q[:,i] = a/R[i][i]
        j = i + 1
        while j <= n-1:
            a = Q[:,i]
            b = Q[:,j].T
            R[i][j] = np.dot(b, a)
            Q[:,j] = b-(R[i][j])*(Q[:,i])
            j += 1
        i += 1
    return Q, R 

In [61]:
Q, R = gram(A)
print(Q)
print(R)

[[ 0.75670886 -0.59354132]
 [ 0.61216924  0.72928996]
 [ 0.1043276   0.30828684]
 [ 0.20434353 -0.1442362 ]]
[[ 0.92760682  0.71950796]
 [ 0.          0.70849023]]


In [62]:
Q,R = la.qr(A, mode="economic") # Use mode="economic" for reduced QR.
print(Q)
print(R)

[[-0.75670886  0.59354132]
 [-0.61216924 -0.72928996]
 [-0.1043276  -0.30828684]
 [-0.20434353  0.1442362 ]]
[[-0.92760682 -0.71950796]
 [ 0.         -0.70849023]]


Our algorithm gives exactly the negative of SciPy's Gram-Schmidt algorithm. This is perfectly fine, as we proved in Problem Set 2 that these two factorizations are identical.

## Problem 2

In [69]:
def gram_det(A):
    return abs((gram(A)[1]).diagonal().prod())

In [70]:
#Output for a square matrix A
A = np.random.random((4,4))
gram_det(A)

%timeit gram_det(A)

The slowest run took 4.41 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 97.6 µs per loop


In [71]:
#Check this determinant manually
np.linalg.det(A)

%timeit np.linalg.det(A)

The slowest run took 98.13 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 11.7 µs per loop


## Problem 3

In [84]:
def solver(A, b):
    """Solves equation of the form Ax=b where A is an nxn matrix
    and b is a vector of length n using QR decomposition"""
    Q = gram(A)[0]
    R = gram(A)[1]
    QT = Q.T
    y = np.dot(QT, b)
    
    #Solve Rx = y for x using back substitution
    n = np.size(y)
    x = np.zeros(n)
    
    x[-1] = (y[-1])/R[-1, -1]
    for i in range(n-2, -1, -1):
        x[i] = (y[i] - np.sum(R[i, i+1:] * x[i+1:]))/R[i,i]
    return x

In [85]:
A = np.random.random((4,4))
b = np.random.random((4,1))
print(A)
print(b)

solver(A,b)

[[ 0.66091854  0.45313202  0.4139155   0.55141113]
 [ 0.27035871  0.0583324   0.08745433  0.35002164]
 [ 0.65276199  0.41572612  0.20805255  0.07086233]
 [ 0.84344145  0.32327592  0.41852578  0.98982664]]
[[ 0.53292117]
 [ 0.73854891]
 [ 0.67050492]
 [ 0.04038322]]


array([-10.6127543 ,  36.04068599, -40.39841297,  14.39475469])

In [86]:
np.linalg.solve(A,b)

array([[-10.6127543 ],
       [ 36.04068599],
       [-40.39841297],
       [ 14.39475469]])

We have verified above that our function gives the same output as the NumPy linalg solver.

## Problem 4: Householder Algorithm

In [89]:
np.eye(2)

array([[ 1.,  0.],
       [ 0.,  1.]])

In [90]:
def sign(x):
    if x >= 0:
        return 1
    else:
        return -1

def householder(A):
    m = np.shape(A)[0]
    n = np.shape(A)[1]
    R = np.copy(A)
    Q = np.eye(m)
    k = 0
    while k <= n-1:
        u = np.copy(R[k:,k])
        u[0] = u[0]+sign(u[0])*np.linalg.norm(u)
        u = u/np.linalg.norm(u)
        uT = u.T
        uTR = np.dot(uT, R[k:,k:])
        R[k:,k:] += -2*np.outer(u, uTR)
        uTQ = np.dot(uT, Q[k:,:])
        Q[k:,:] += -2*np.outer(u, uTQ)
        k+=1
    return Q.T, R 

In [91]:
A = np.random.random((5, 3))
Q,R = la.qr(A)
print(A)
print(Q)
print(R)

Q, R = householder(A)
print(Q)
print(R)

[[ 0.34872273  0.65997485  0.84679209]
 [ 0.68771354  0.95043757  0.82285859]
 [ 0.74026871  0.23086033  0.76699056]
 [ 0.67499975  0.37977192  0.21999355]
 [ 0.13315458  0.66809619  0.97365066]]
[[-0.27432896 -0.40784396 -0.18845387 -0.82637912  0.19997404]
 [-0.54100215 -0.40416276  0.41116241  0.14111472 -0.59581981]
 [-0.58234561  0.46783594 -0.6286109   0.05512847 -0.20931391]
 [-0.53100062  0.23367358  0.4546406   0.11828551  0.66539121]
 [-0.10474843 -0.62995394 -0.43997826  0.52929536  0.34416763]]
[[-1.27118449 -1.10132064 -1.34292653]
 [ 0.         -0.87742063 -0.88105047]
 [ 0.          0.         -0.63175848]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
[[-0.27432896 -0.40784396 -0.18845387 -0.82637912  0.19997404]
 [-0.54100215 -0.40416276  0.41116241  0.14111472 -0.59581981]
 [-0.58234561  0.46783594 -0.6286109   0.05512847 -0.20931391]
 [-0.53100062  0.23367358  0.4546406   0.11828551  0.66539121]
 [-0.10474843 -0.62995394 -0.43997826  0

Our Householder function works although there are some rounding errors for values close to 0.

## Problem 5: Hessenberg Transformation

In [95]:
def hessenberg(A):
    m = np.shape(A)[0]
    n = np.shape(A)[1]
    H = np.copy(A)
    Q = np.eye(m)
    k = 0
    while k <= n-3:
        u = np.copy(H[k+1:,k])
        u[0] = u[0]+sign(u[0])*np.linalg.norm(u)
        u = u/np.linalg.norm(u)
        uT = u.T
        uTH = np.dot(uT, H[k+1:,k:])
        H[k+1:,k:] += -2*np.outer(u, uTH)
        
        HU = np.dot(H[:,k+1:],u)
        H[:,k+1:] += -2*np.outer(HU, uT)
        
        uTQ = np.dot(uT, Q[k+1:,:])
        Q[k+1:,:] += -2*np.outer(uT, uTQ)
        
        k+=1
    return H, Q.T

In [96]:
A = np.random.random((4, 4))
H, Q = la.hessenberg(A, calc_q=True)
print(A)
print(H)
print(Q)

H, Q = hessenberg(A)
print(H)
print(Q)

[[ 0.78502425  0.65789695  0.29186986  0.49769646]
 [ 0.25256935  0.16518512  0.88297513  0.79713334]
 [ 0.42149956  0.99350072  0.57825134  0.2953125 ]
 [ 0.93412458  0.28521217  0.37040322  0.31259551]]
[[ 0.78502425 -0.71445865 -0.47295845 -0.1777001 ]
 [-1.05548183  0.99033706  0.62581623 -0.19349675]
 [ 0.          0.99989765  0.33058907 -0.35314666]
 [ 0.          0.         -0.67336518 -0.26489414]]
[[ 1.          0.          0.          0.        ]
 [ 0.         -0.23929294 -0.86072574 -0.44932182]
 [ 0.         -0.39934327 -0.33456634  0.85357502]
 [ 0.         -0.88502195  0.38368812 -0.26366566]]
[[  7.85024252e-01  -7.14458650e-01  -4.72958450e-01  -1.77700099e-01]
 [ -1.05548183e+00   9.90337058e-01   6.25816225e-01  -1.93496753e-01]
 [ -5.55111512e-17   9.99897646e-01   3.30589066e-01  -3.53146663e-01]
 [ -1.11022302e-16   3.33066907e-16  -6.73365182e-01  -2.64894144e-01]]
[[ 1.          0.          0.          0.        ]
 [ 0.         -0.23929294 -0.86072574 -0.44932182

Our output is the same with some rounding errors.